In [53]:
import pandas as pd
import numpy as np
import pdb
from datetime import timedelta
import os
import dask.dataframe as dd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
dd.read_csv('data/train.csv').head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,2013-01-01,25,103665,7.0,NaN
1,1,2013-01-01,25,105574,1.0,NaN
2,2,2013-01-01,25,105575,2.0,NaN
3,3,2013-01-01,25,108079,1.0,NaN
4,4,2013-01-01,25,108701,1.0,NaN


In [9]:
dd.read_csv('data/test.csv').head()

,id,date,store_nbr,item_nbr,onpromotion
0,125497040,2017-08-16,1,96995,False
1,125497041,2017-08-16,1,99197,False
2,125497042,2017-08-16,1,103501,False
3,125497043,2017-08-16,1,103520,False
4,125497044,2017-08-16,1,103665,False


# Preprocessing Train Data

In [10]:
# STEP 1: takes a few minutes to run
dtypes = {'id':'uint32', 'item_nbr':'int32', 'store_nbr':'int8', 'unit_sales':'float32', 'onpromotion':np.bool}

train = pd.read_csv('data/train.csv', usecols=[1,2,3,4,5], dtype=dtypes, parse_dates=['date'],
                    skiprows=range(1, 86672217) #Skip dates before 2016-08-01
                    )

train.onpromotion.fillna(False,inplace=True)
train.loc[(train.unit_sales<0),'unit_sales'] = 0 # eliminate negatives
train['unit_sales'] =  train['unit_sales'].apply(pd.np.log1p) #logarithm conversion
train['dow'] = train['date'].dt.dayofweek

# creating records for all items, in all markets on all dates
# for correct calculation of daily unit sales averages.
u_dates = train.date.unique()
u_stores = train.store_nbr.unique()
u_items = train.item_nbr.unique()
train.set_index(['date', 'store_nbr', 'item_nbr'], inplace=True)
train = train.reindex(
    pd.MultiIndex.from_product(
        (u_dates, u_stores, u_items),
        names=['date','store_nbr','item_nbr']
    )
)

del u_dates, u_stores, u_items

train.loc[:, 'unit_sales'].fillna(0, inplace=True) # fill NaNs
train.reset_index(inplace=True) # reset index and restoring unique columns  

# Rolling Mean Generator 

In [66]:
def run_rolling_mean(train,test):
    
    test_start = test.date.min()
    test_end = test.date.max()
    
    #Days of Week Means
    ma_dw = train[['item_nbr','store_nbr','dow','unit_sales']].groupby(['item_nbr','store_nbr','dow'])['unit_sales'].mean().to_frame('madw')
    ma_dw.reset_index(inplace=True)
    ma_wk = ma_dw[['item_nbr','store_nbr','madw']].groupby(['store_nbr', 'item_nbr'])['madw'].mean().to_frame('mawk')
    ma_wk.reset_index(inplace=True)

    #Moving Averages
    ma_is = train[['item_nbr','store_nbr','unit_sales']].groupby(['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('mais226')
    for i in [112,56,28,14,7,3,1]:
        tmp = train[train.date>test_start-timedelta(int(i))]
        tmpg = tmp.groupby(['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('mais'+str(i))
        ma_is = ma_is.join(tmpg, how='left')

    del tmp,tmpg,train

    ma_is['mais']=ma_is.median(axis=1)
    ma_is.reset_index(inplace=True)

    #Format and Merge Averages with Test
    test['dow'] = test['date'].dt.dayofweek
    test = pd.merge(test, ma_is, how='left', on=['item_nbr','store_nbr'])
    test = pd.merge(test, ma_wk, how='left', on=['item_nbr','store_nbr'])
    test = pd.merge(test, ma_dw, how='left', on=['item_nbr','store_nbr','dow'])

    del ma_is, ma_wk, ma_dw

    #Forecasting Test
    test['unit_sales'] = test.mais 
    pos_idx = test['mawk'] > 0
    test_pos = test.loc[pos_idx]
    test.loc[pos_idx, 'unit_sales'] = test_pos['mais'] * test_pos['madw'] / test_pos['mawk']
    test.loc[:, "unit_sales"].fillna(0, inplace=True)
    test['unit_sales'] = test['unit_sales'].apply(pd.np.expm1) # restoring unit values 

    #50% more for promotion items
    test.loc[test['onpromotion'] == True, 'unit_sales'] *= 1.5

    test.to_csv('data/generated/train_{}_RM.csv.gz'.format(test_start.strftime("%m-%d-%y")), \
                index=False, \
                float_format='%.3f', \
                compression='gzip')

# Run

In [67]:
# STEP 2: Set cross validation windows

day_offset = 14 # size of window (in days) for cv set
n_windows = 18 # how many windows backward to generate

cv_end = pd.to_datetime('2017-08-16') # aka training set start
cv_start = cv_end - pd.Timedelta((n_windows * day_offset),'D')
cv_windows = pd.date_range(cv_start,cv_end,freq='{}D'.format(day_offset))

In [ ]:
# STEP 3: Loop over cv windows and generate features

for idx, test_start in enumerate(cv_windows):
    sub_train = train[train.date < test_start]
    sub_test = train[(train.date >= test_start) & (train.date <= cv_periods[idx+1])]
    run_rolling_mean(sub_train,sub_test)

In [ ]:
# OUTPUT: will be a compressed csv with rolling features for each cv window